# Creo un nuovo dataset 


---
## Analisi dei commenti di un ristorante

### Old Wild West - Padova

URL di riferimento: https://www.tripadvisor.it/Restaurant_Review-g187867-d3346323-Reviews-Old_Wild_West-Padua_Province_of_Padua_Veneto.html

Home --> Ristoranti --> Padova

In [1]:
# IMPORT NECESSARI
import requests
from bs4 import BeautifulSoup

print("Imported")

Imported


In [2]:
page_URL = "https://www.tripadvisor.it/Restaurant_Review-g187867-d3346323-Reviews-Old_Wild_West-Padua_Province_of_Padua_Veneto.html"

data = requests.get(page_URL)
clear_data = BeautifulSoup(data.content, "lxml")

print("Done")

Done


---
### STRUTTURA HTML DEL SITO 

#### REVIEW
`<p class="partial_entry"> qui c'è la review </p>`

#### N° TOTALE DI REVIEWS
`<a data-page-number="25" ...>
    25
 </a>`
 
#### NAMES
`<span class="expand_inline scrname" …>AlexandraCapeTown</span>`

#### RATINGS
`<span class="ui_bubble_rating bubble_30"></span>`

---
### PRIMO SCRAPING (sulla prima pagina)

Ottengo i nomi, i commenti, le review e da dove è stato scritto il commento per quanto rigurda la prima pagina. Ciò è necessario, in quanto, dalla seconda in poi l'URL cambia, permettendoci di iterare.

In [3]:
# OTTENGO UNA LISTA DI DIV CHE CONTENGONO NOME, RECENSIONE E VOTO

all_reviews = []
all_names = []
all_ratings = []
all_mobile = []

review_container = clear_data.findAll("div", attrs={"class":"review-container"})

for container in review_container:
  
    ### REVIEW ###
    review = container.find('p', attrs={'class': 'partial_entry'})
    if review:
        all_reviews.append(review.text)
        
    ### NAME ###
    name = container.find('span', attrs={'class': ('expand_inline', 'scrname')})
    if name:
      all_names.append(name.text)
    else:
      all_names.append('NO_NAME')
    
    ### RATING ###
    rating = container.find('span', attrs={'class': 'ui_bubble_rating'})
    all_ratings.append(rating['class'][1][7:8])
    
    ### MOBILE ###
    mobile = container.find('span', attrs={'class': 'viaMobile'})
    if mobile:
      all_mobile.append('Mobile')
    else:
      all_mobile.append('PC')
    
    
################################################################################################################################
        
print("Executed")
print('Nomi: ', len(all_names))
print('Commenti: ', len(all_reviews))
print('Voti: ', len(all_ratings))
print('Scritto da: ', len(all_ratings))

Executed
Nomi:  10
Commenti:  10
Voti:  10
Scritto da:  10


### SECONDO SCRAPING (dalla seconda pagina in poi)

Notiamo dall'URL che è possibile iterare sulle pagine, ottenendo i dati!

In [4]:
from tqdm import tqdm

URL_COMPLETO = 'https://www.tripadvisor.it/Restaurant_Review-g187867-d3346323-Reviews-Old_Wild_West-Padua_Province_of_Padua_Veneto.html'

URL_p1 = 'https://www.tripadvisor.it/Restaurant_Review-g187867-d3346323-Reviews-or'
URL_p2 = '0-Old_Wild_West-Padua_Province_of_Padua_Veneto.html'
    
# OTTENGO I COMMENTI, NOMI E VOTI DALLA SECONDA PAGINA IN POI

total_pages = int(clear_data.select("a.pageNum.last.taLnk")[0].text) # 25 come intero

# per iterare dobbiamo partire dalla seconda pagina dei commenti, così nell'URL compaiono le cifre che ci servono
for r in tqdm(range(1,total_pages)):

    # OTTENGO L'URL DELLA PAGINA
    URL_temp = URL_p1 + str(r) + URL_p2

    # OTTENGO I DATI E CI APPLICO BEATIFULSOUP
    data = requests.get(URL_temp)
    clear_data = BeautifulSoup(data.content, "lxml")
    

    review_container = clear_data.findAll("div", attrs={"class":"review-container"})

    for container in review_container:
  
      ### REVIEW ###
      review = container.find('p', attrs={'class': 'partial_entry'})
      if review:
          all_reviews.append(review.text)

      ### NAME ###
      name = container.find('span', attrs={'class': ('expand_inline', 'scrname')})
      if name:
        all_names.append(name.text)
      else:
        all_names.append('NO_NAME')

      ### RATING ###
      rating = container.find('span', attrs={'class': 'ui_bubble_rating'})
      all_ratings.append(rating['class'][1][7:8])

      ### MOBILE ###
      mobile = container.find('span', attrs={'class': 'viaMobile'})
      if mobile:
        all_mobile.append('Mobile')
      else:
        all_mobile.append('PC')
    
        
################################################################################################################################
        
print("Executed")
print('total_pages: ', total_pages)
print('Nomi: ', len(all_names))
print('Commenti: ', len(all_reviews))
print('Voti: ', len(all_ratings))
print('Scritto da: ', len(all_mobile))

100%|██████████| 41/41 [00:44<00:00,  1.01it/s]

Executed
total_pages:  42
Nomi:  416
Commenti:  416
Voti:  416
Scritto da:  416


In [5]:
# display(all_reviews)

In [6]:
# print(all_names)

In [7]:
#print(all_ratings, '\n')
#print(len(all_ratings))

In [8]:
#display(all_mobile)

#### Problema recensioni troppo lunghe
Alla fine di alcune recensioni troviamo un '...Più', che non possiamo espandere. Data la mole di dati, decidiamo dunque di non considerarle.

In [9]:
all_reviews_filtered = []
counter_reviews_scartate = 0;
counter_reviews_da_usare = 0;

for review in all_reviews:
    if '...Più' in review:
        all_reviews_filtered.append('RECENSIONE_TROPPO_LUNGA')
        counter_reviews_scartate+=1;
    else:
        all_reviews_filtered.append(review)
        counter_reviews_da_usare+=1;
        
        
print(
    'Totale recensioni:', counter_reviews_scartate + counter_reviews_da_usare, '\n',
    'Recensioni analizzabili:', counter_reviews_da_usare, '\n',
    'Recensioni scartate:', counter_reviews_scartate, '\n')

display(all_reviews_filtered[:10])

Totale recensioni: 416 
 Recensioni analizzabili: 217 
 Recensioni scartate: 199 



['Bel locale, personale accogliente e gentile, qualche difficoltà per il parcheggio, ottima carne, cucinata a puntino, ottime patate, buona birra. Forse un pò caro, ma ci sta....',
 'RECENSIONE_TROPPO_LUNGA',
 'RECENSIONE_TROPPO_LUNGA',
 'RECENSIONE_TROPPO_LUNGA',
 'RECENSIONE_TROPPO_LUNGA',
 'cucina varia ed eccellente, in grado di accontentare, in locale rustico dalla calda atmosfera, la prevalente clientela giovanile che lo frequenta ma anche coppie o gruppi di amici meno giovani alla ricerca di novità culinarie che si scostino dalla routine',
 'Ormai è una certezza, ci vengo da molti anni. Servizio e qualità buona. Consigliato per chi vuole gustare qualcosa di particolare. Ambiente giovanile e adatto anche a famiglie con bambini. Consigliato',
 'RECENSIONE_TROPPO_LUNGA',
 'RECENSIONE_TROPPO_LUNGA',
 'Ultimamente non sono organizzati. Tempi di attesa lunghissimi . Dopo un ora arriva il pranzo senza posate . Atri 15 minuti ad aspettare le posate.']

---
### Analisi dei nomi
Cerchiamo di capire se chi ha commentato è un uomo (M) o una donna (F).

#### Elimino spazi e numeri
Ho bisogno di tenere solo le informazioni riguardanti il nome, elimino il resto.

In [10]:
import re

all_names_filtered = []

regex = re.compile('[^a-zA-Z]')

for name in all_names:
    
    if ' ' in name:                           # se c'è uno spazio nel nome
        space = name.index(' ')               # ne trovo l'indice
        name = regex.sub('', name[:space])    # effettuo un controllo su tutto ciò che non è alfabeto fino a quell'indice
        all_names_filtered.append(name)       # e aggiungo la stringa
        
    else:
        name = regex.sub('', name)            # in caso contrario effettuo un controllo su tutto ciò che non è alfabeto
        all_names_filtered.append(name)       # e aggiungo la stringa
        
print(all_names, '\n')
print(all_names_filtered)

['giuseppe v', 'I6493YValessandros', 'A8059FHmichelev', 'agostino m', 'Paolooloap95', 'Fiorentin L', 'Voyager22625929988', 'ElyT510', '146irenez', 'Z1873LEgiorgiac', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'gescarolo', 'Virginie F', 'Cinzia D', 'Andrea D', 'Uriel M', 'Sarawanderlust91', 'Geppovr', 'Pierazzarita', 'W9241OLandreac', 'chiaracaterinad2016', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'Gab901', 'corrado v', 'Vicky Z', 'federicoferro', 'Azzurra S', 'MARIO P', 'Albe72xx', '714filippog', 'Toni C', 'Lucia F', 'ebensi', 'distalbere66', 'damzamb', 'Massimo L', 'Costanza G', 'AlessiaDeFrancesch', 'CHIARAC525', 'GiuliaTartaglia', 'BoardingPass418964', 'barbaralB1401MD', 'marty1190', 'Alessia D', '869AndreaF', 'Letizia S', 'Chips_and_Pepper', 'Carlettoo', 'SOMSYAN', 'ItalianTourist68', 'alezac79', 'francescop1961-', 'NO_NAME', 'NO_NAME', 'NO_NAME', 

#### Utilizzo gender_guesser
Infine, posso utilizzare la libreria gender_guesser.

NB: Genderize ti permette di analizzare 1000 nomi al giorno.

In [11]:
!pip install gender_guesser

In [12]:
import gender_guesser.detector as gender
d = gender.Detector()

names_with_gender = []

for name in all_names_filtered:
    temp_gender = d.get_gender(name)
    names_with_gender.append(temp_gender)
  
print(names_with_gender)

['unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'female', 'female', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'andy', 'unknown', 'female', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'male', 'female', 'unknown', 'unknown', 'unknown', 'male', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'male', 'male', 'unknown', 'female', 'unknown', 'unknown', 'male', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 

In [13]:
all_genders = []
i = 0

for guess in names_with_gender:
    if guess == 'male' or guess == 'mostly_male':
        all_genders.append('M')
    elif guess == 'female' or guess == 'mostly_female':
        all_genders.append('F')
    else:
        all_genders.append('UNKNOWN')

    
print(all_genders)

['UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'F', 'F', 'F', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'F', 'UNKNOWN', 'F', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'F', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'F', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'F', 'UNKNOWN', 'F', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'M', 'M', 'UNKNOWN', 'F', 'UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'F', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'F', 'F', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'F', 'UNKNOWN', 'UNKNOWN', 'UN

---
## Salviamo ora i dati ottenuti in una tabella Excel
Così non dovremo ogni volta rieseguire tutto il codice



### Controlliamo che ci siano lo stesso numero di dati
Potremo così creare una tabella sfruttando Pandas

In [14]:
print('Nomi: ', len(all_names_filtered))
print('Commenti: ', len(all_reviews_filtered))
print('Voti: ', len(all_ratings))
print('Mobile: ', len(all_mobile))

Nomi:  416
Commenti:  416
Voti:  416
Mobile:  416


### Eliminiamo le recensioni troppo lunghe e quelle di cui non sappiamo il genere del commentatore
Creo una lista di liste [ ['NOME', 'GENERE', 'RECENSIONE', 'VOTO'], ['NOME', 'GENERE', 'RECENSIONE', 'VOTO'], ... ]

In [15]:
complete_list = []

i = len(all_names_filtered)

for indice in range(i):
  if all_genders[indice] != 'UNKNOWN' and all_reviews_filtered[indice] != 'RECENSIONE_TROPPO_LUNGA':
    temp = [all_names_filtered[indice], all_genders[indice], all_reviews_filtered[indice], all_ratings[indice], all_mobile[indice]]
    complete_list.append(temp)
    
print('DATI ANALIZZABILI: ', len(complete_list))
display(complete_list[:5])


DATI ANALIZZABILI:  51


[['Cinzia',
  'F',
  'Pagare trenta euro per due hamburgher mediocri e due birre mi sembra eccessivo. Specialmente se nel menù gli hamburgher vengono descritti come freschi e invece sono precotti.',
  '2',
  'Mobile'],
 ['Andrea',
  'F',
  "Abbiamo cenato in tre per lavoro un locale carino ma non climatizzato abbiamo patito parecchio il caldo.\nL'attesa è stata lunga rispetto al solito (vado spesso a mangiare all'old a torino)\nMa comunque la cena e piaciuta a tutti molto buono.\nPersonale molto gentile.",
  '3',
  'Mobile'],
 ['Uriel',
  'M',
  'Nervi nel chili con carne, patatine mezze bruciate. Abbiamo speso 40euro in due e avevamo ancora fame',
  '1',
  'Mobile'],
 ['Vicky',
  'F',
  'Mi aspettavo un hamburger particolare da una catena di questo tipo invece ho mangiato un piatto estremamente normale e classico .L’ambiente è a tema con il food e il prezzo adeguato.',
  '4',
  'Mobile'],
 ['Azzurra',
  'F',
  'Tappa settimanale, buonissimo, velocissimo e mediamente economico! Super c

### Creiamo la tabella

In [16]:
# !pip install pandas
# !pip install xlsxwriter
import pandas as pd

In [17]:
# Create a Pandas dataframe from the data.
df = pd.DataFrame({'Nomi': [elemento[0] for elemento in complete_list], 
                   'Genere': [elemento[1] for elemento in complete_list], 
                   'Recensioni': [elemento[2] for elemento in complete_list],
                   'Voti': [elemento[3] for elemento in complete_list],
                   'Scritta da': [elemento[4] for elemento in complete_list]})

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('RISTORANTE_20.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='Sheet1')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [18]:
partial_dataframe = pd.read_excel("RISTORANTE_20.xlsx")

display(partial_dataframe)

,Nomi,Genere,Recensioni,Voti,Scritta da
0,Cinzia,F,Pagare trenta euro per due hamburgher mediocri...,2,Mobile
1,Andrea,F,Abbiamo cenato in tre per lavoro un locale car...,3,Mobile
2,Uriel,M,"Nervi nel chili con carne, patatine mezze bruc...",1,Mobile
3,Vicky,F,Mi aspettavo un hamburger particolare da una c...,4,Mobile
4,Azzurra,F,"Tappa settimanale, buonissimo, velocissimo e m...",5,Mobile
5,Lucia,F,Bel posto .. ma per il galletto non ci siamo p...,2,Mobile
6,Costanza,F,"Siamo andati in gruppo il sabato sera, ma l'at...",3,Mobile
7,Marco,M,"Servizio molto veloce, cibo molto buoni e cuci...",5,Mobile
8,Elena,F,Siamo stati a cena qui ieri sera perché eravam...,4,Mobile
9,Emanuele,M,Trovandomi a Padova per controlli medici mi fe...,3,PC
